In [5]:
# importing the zipfile module
from zipfile import ZipFile

# loading the temp.zip and creating a zip object
with ZipFile("processed-images.zip", 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(
        path="")

In [6]:
import os
import pandas as pd
import numpy as np

# Load the CSV data
df = pd.read_csv('images.csv', header=None, names=["image_path", "label", "timestamp", "moon_phase", "wind", "weather", "temp", "humidity", "pressure"])

# Filter out rows where image paths do not exist
df = df[df['image_path'].apply(os.path.exists)]



In [7]:
# Convert timestamp to datetime with UTC conversion, with error handling
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)

# Drop rows with invalid timestamps (NaT)
df = df.dropna()

# Extract features from the timestamp
df['hour'] = df['timestamp'].dt.hour  # Hour of the day
df['day_of_week'] = df['timestamp'].dt.dayofweek  # Day of the week
df['is_night'] = (df['hour'] < 6) | (df['hour'] > 18)  # Simple binary night/day indicator

# Handle categorical features (moon_phase, weather) with one-hot encoding
df = pd.get_dummies(df, columns=['moon_phase', 'weather'], drop_first=True)


<ipython-input-7-bc1245c452ca>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['timestamp'].dt.hour  # Hour of the day
<ipython-input-7-bc1245c452ca>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = df['timestamp'].dt.dayofweek  # Day of the week
<ipython-input-7-bc1245c452ca>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [8]:
# Extract features as a NumPy array
metadata_columns = ['hour', 'day_of_week', 'is_night', 'wind', 'temp', 'humidity', 'pressure'] + \
                   [col for col in df.columns if col.startswith('moon_phase_') or col.startswith('weather_')]
metadata = df[metadata_columns].values  # Convert to NumPy array
metadata = metadata.astype(np.float32)  # Ensure float32 dtype

# Convert the label to a NumPy array (binary classification: deer or not-deer)
labels = (df['label'] == 'deer').astype(int).values  # Convert to 0 or 1 (0: not deer, 1: deer)


In [9]:
from tensorflow.keras.preprocessing import image

# Image loading function with normalization
def load_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array.astype(np.float32)  # Ensure it's in float32 format

# Load all images in parallel using ThreadPoolExecutor
def load_images_in_parallel(image_paths, batch_size=32, target_size=(224, 224)):
    images = []
    with ThreadPoolExecutor() as executor:
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i + batch_size]
            batch_images = list(executor.map(lambda path: load_image(path, target_size), batch_paths))
            images.extend(batch_images)
    return np.array(images)

# Load images and convert to NumPy array
images = load_images_in_parallel(df['image_path'].values)


In [10]:
# Ensure correct shape and dtype
print(images.shape, images.dtype)  # Should be (num_samples, 224, 224, 3) and float32
print(metadata.shape, metadata.dtype)  # Should be (num_samples, N) and float32 (N depends on one-hot encoding)
print(labels.shape, labels.dtype)  # Should be (num_samples,) and int32


(2129, 224, 224, 3) float32
(2129, 16) float32
(2129,) int64


In [11]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train_images, X_val_images, X_train_metadata, X_val_metadata, y_train, y_val = train_test_split(
    images, metadata, labels, test_size=0.2, random_state=42
)


In [12]:
from tensorflow.keras import layers, models, optimizers
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf

# Define the model with dropout and learning rate scheduler
image_input = layers.Input(shape=(224, 224, 3))  # Shape of the images (224x224 RGB images)
metadata_input = layers.Input(shape=(metadata.shape[1],))  # Shape of metadata (after one-hot encoding)

# Image model
x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)

# Combine image model and metadata
combined = layers.concatenate([x, metadata_input])

# Add a fully connected layer, dropout for regularization, and output layer
x = layers.Dense(128, activation='relu')(combined)
x = layers.Dropout(0.5)(x)  # Dropout to reduce overfitting
x = layers.Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

# Define the model
model = models.Model(inputs=[image_input, metadata_input], outputs=x)

# Compile the model with a learning rate scheduler
initial_lr = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_lr, decay_steps=10000, decay_rate=0.9, staircase=True
)

optimizer = optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
from sklearn.metrics import classification_report

# Train the model
history = model.fit(
    [X_train_images, X_train_metadata],  # Input data (images and metadata)
    y_train,  # Labels
    validation_data=([X_val_images, X_val_metadata], y_val),  # Validation data
    epochs=20,
    batch_size=32
)

# Model evaluation
val_loss, val_acc = model.evaluate([X_val_images, X_val_metadata], y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")

# Predictions and classification report
y_pred = (model.predict([X_val_images, X_val_metadata]) > 0.5).astype(int)  # Convert predictions to 0 or 1

# Classification report (Precision, Recall, F1-Score)
print("\nClassification Report:")
print(classification_report(y_val, y_pred))


Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6315 - loss: 0.7895

KeyboardInterrupt: 